## An ECB/CBC detection oracle

Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like: 

```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```

Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use `rand(2)` to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.


In [1]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from os import urandom
from random import randint

https://docs.python.org/3/library/random.html#random.randint

In [2]:
# Returns 5-10 random bytes (count chosen randomly).
def random_bytes():
    count = randint(5, 10)
    return urandom(count)

CBC mode API (Pycryptodome) already creates a random IV, see https://www.pycryptodome.org/src/cipher/classic#cbc-mode.

In [3]:
def encryption_oracle(plaintext):
    key = urandom(AES.block_size) # Generate a random AES key.
    # Append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.
    plaintext = random_bytes() + plaintext + random_bytes()
    
    # Encrypt under ECB 1/2 the time, and under CBC the other half.
    cipher = None
    if randint(0, 1) == 0:
        cipher = AES.new(key, AES.MODE_ECB)
    else:
        cipher = AES.new(key, AES.MODE_CBC)

    padded = pad(plaintext, cipher.block_size)
    return cipher.encrypt(padded)

In [5]:
test = encryption_oracle(b'test')
test

b'GG/\xce\x94\xc7c\xe0\xfc\xf8\xa1\x88W\xb9]\xcep]\x827\x12\x9e\x93P\xaa\xc2%W\xbc\x10\xea\xc5'

In [6]:
# Returns (is_ecb:boolean, counts:dict) tuple, given ciphertext.
def detect_ecb(ciphertext):
    assert len(ciphertext) % AES.block_size == 0
    counts = {}
    for i in range(0, len(ciphertext), AES.block_size):
        block = ciphertext[i:i + AES.block_size]
        if block not in counts:
            counts[block] = 0
        counts[block] += 1
    return max(counts.values()) > 1, counts

In [7]:
is_ecb, counts = detect_ecb(test.hex())
is_ecb

False